In [5]:
# Manejo de archivos
import os
import cv2
import shutil

# Interfaz
import argparse
from termcolor import cprint

# Manejo de Datos
import numpy as np
import pandas as pd

# Agrupamiento
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

# Procesamiento de los datos
El dataset que se enviará al algoritmo de `KMeans` o `AgglomerativeClustering` tendrá la siguiente forma

| name | image | 
|-------|------|
| 1.jpg | [2,3,4,5,10,0] |
| 2.jpg | [2,0,41,5,10,0] |

El vector asociado a cada imagen representa su información y características principales, la forma de obtener este vector es indistinta para describir esto. El representar las imágenes como su vector de características hará que sea mucho más fácil de agrupar, pues los algoritmos agrupan por características.

## Primer modo de procesar la imagen 
El primer modo carga la imagen HSV y la reduce para que en esta se represente más información por píxel y así pueda ser agrupada.

![Image](docs/2022-09-27_13-04.jpg)

```python
L = 9

image = cv2.imread("./data/007.jpeg")
cv2.imshow("Original",image)

processed = cv2.cvtColor(image,cv2.COLOR_BGR2HLS)
cv2.imshow("HSV",processed)

processed = cv2.resize(processed,(L,L))
processed = processed[:,:,0]
cv2.imshow("Resized",processed)

cv2.waitKey()

cv2.destroyAllWindows()

processed = processed.reshape((L*L))
processed
```

In [ ]:
# Constantes
shape = (9,9)
save_on_csv = False

In [22]:
def image2VectorA(file):
    image = cv2.imread(file)
    processed = cv2.cvtColor(image,cv2.COLOR_BGR2HLS)
    processed = cv2.resize(processed,shape)
    processed = processed[:,:,0]
    processed = processed.reshape(shape[0]*shape[1])
    return processed

image2VectorA("data/007.jpeg")

array([170, 170, 170, 170, 170, 170, 170, 170, 170, 170,  83, 121, 165,
       170, 170,  10,  24,  99,  21,  70,  51, 112,   7, 170,  61,  73,
        72,  58,  18,  49,  34, 123, 170, 123,  42, 147, 121, 138, 173,
        73,  49, 170,  27, 102,  37, 117,  50,  52,  37, 154, 170, 145,
       117, 101, 174, 162, 102, 157,  25,  98,   4, 130, 142,  32,  26,
        23,  34,  27, 170,  15,  33, 119, 170, 170, 170, 170, 170, 170,
       170, 170, 170], dtype=uint8)

# Segunda forma
Se puede usar el histograma para realizar este procesamiento.

In [24]:
def image2VectorB(file):
    image = cv2.imread(file)
    image = image[:,:,0]
    hist,bins = np.histogram(image.ravel(),360,[0,360])
    return hist

image2VectorB("data/007.jpeg")

array([  756,   172,   210,   223,   253,   335,   318,   500,   615,
       12949,   721,   558,   381,   427,   350,   319,   324,   280,
         303,   301,   253,   253,   257,   260,   235,   226,   208,
         245,   255,   234,   207,   218,   220,   232,   211,   220,
         210,   199,   197,   208,   195,   191,   197,   213,   163,
         205,   200,   194,   172,   193,   183,   205,   193,   192,
         196,   197,   193,   188,   206,   210,   181,   213,   200,
         211,   181,   178,   190,   218,   205,   184,   199,   173,
         168,   188,   162,   168,   179,   175,   159,   140,   153,
         151,   129,   145,   169,   108,   141,   148,   118,   124,
         112,   104,   105,   109,   112,   104,   101,   100,    98,
          90,    85,   102,    90,    87,    75,   107,    86,    99,
          79,    83,    88,    60,    73,    70,    72,    81,    81,
          84,    71,    57,    83,    49,    63,    69,    60,    84,
          65,    50,

In [26]:
def load_data(dir):
    names = []
    images = []

    if not os.path.exists(dir):
        return []
    
    os.chdir(dir)

    for file in os.listdir("."):
        try :
            names.append(file)
            images.append(image2VectorB(file))
        except:
            print(f"{file} is not an Image")
        finally:
            pass

    return names,images

names,images = load_data("data")

In [31]:
def cluster(method,names,data,n_clusters=1):
    if method == "kmeans":
        model = KMeans(n_clusters,init="random")
        model.fit(data)
        dic = {"file":names,"cluster":model.labels_}
        df = pd.DataFrame(dic)
        return df

df = cluster("kmeans",names,images,4)
df

,file,cluster
0,660.jpg,2
1,705.jpg,3
2,1219.jpg,0
3,153.jpg,0
4,1236.jpg,3
...,...,...
1395,274.jpg,1
1396,902.jpeg,1
1397,242.jpeg,0
1398,264.jpg,2


In [28]:
def group_dir(df):
    for file,cluster in df.values:
        cluster_dir = f"Cluster_{cluster}"

        if not os.path.exists(cluster_dir):
            os.mkdir(cluster_dir)

        print(f"{file} -> {cluster_dir}/{file}")
        shutil.move(file,cluster_dir)

group_dir(df)

660.jpg -> Cluster_1/660.jpg
705.jpg -> Cluster_0/705.jpg
1219.jpg -> Cluster_2/1219.jpg
153.jpg -> Cluster_2/153.jpg
1236.jpg -> Cluster_0/1236.jpg
387.jpg -> Cluster_0/387.jpg
531.jpg -> Cluster_3/531.jpg
1319.jpg -> Cluster_1/1319.jpg
612.jpg -> Cluster_3/612.jpg
791.jpg -> Cluster_0/791.jpg
1259.jpg -> Cluster_0/1259.jpg
425.jpg -> Cluster_3/425.jpg
1368.jpg -> Cluster_3/1368.jpg
315.jpg -> Cluster_2/315.jpg
1005.jpg -> Cluster_1/1005.jpg
810.jpg -> Cluster_3/810.jpg
1186.jpeg -> Cluster_3/1186.jpeg
1227.jpg -> Cluster_3/1227.jpg
580.jpg -> Cluster_2/580.jpg
017.jpg -> Cluster_0/017.jpg
089.jpg -> Cluster_3/089.jpg
167.jpg -> Cluster_0/167.jpg
558.jpg -> Cluster_0/558.jpg
1018.jpg -> Cluster_3/1018.jpg
1053.jpg -> Cluster_0/1053.jpg
955.jpg -> Cluster_2/955.jpg
230.jpeg -> Cluster_3/230.jpeg
716.jpg -> Cluster_1/716.jpg
689.jpg -> Cluster_2/689.jpg
1384.jpg -> Cluster_0/1384.jpg
1148.jpg -> Cluster_3/1148.jpg
1129.jpg -> Cluster_3/1129.jpg
1208.jpg -> Cluster_0/1208.jpg
272.jpg -> 

In [30]:
!pwd
!python ~/Software/File\ Utils/recursive_extract.py .
!rmdir *
!rm *.csv

/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Cluster_3/531.jpg->.
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Cluster_3/612.jpg->.
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Cluster_3/425.jpg->.
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Cluster_3/1368.jpg->.
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Cluster_3/810.jpg->.
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Cluster_3/1186.jpeg->.
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Cluster_3/1227.jpg->.
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Cluster_3/089.jpg->.
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Cluster_3/1018.jpg->.
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Cluster_3/230.jpeg->.
/home/fercho/ESCOM/AM/Actividades/PR01/ML_Textil_Clustering/data/Clus